In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import photutils
import numpy as np
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture
from scipy.optimize import curve_fit

In [ ]:
def unravel(arr):
    '''This function takes a (2D) array as input removes rows of data containing nan. It then flattens the array into a 1D array and returns it.
    The function is called in function wcsplot and the array is used to calculate the values of vmin and vmax.
    Example: If arr is [ nan -0.0403614  -0.03563905 nan nan
                        -0.02530874 -0.01923032 -0.03322243 -0.0152488  -0.02228408
                        nan nan nan nan nan]
    (Each line is a row of arr)
    The array returned is [-0.02530874 -0.01923032 -0.03322243 -0.0152488  -0.02228408] '''
    nan_rows = np.isnan(arr)
    dataclean=arr[~nan_rows]
    dataclean=np.ravel(dataclean)
    return dataclean

In [ ]:
def wcsplot(imgdata,w):
    '''This function is used to plot the FITS file image data with the World Coordinate System instead of pixel coordinates.
    The first parameter is an array containing image data (of NGC 104) and w is a WCS object based on the header information of the FITS file.
    The function plots this data and returns the values of vmin and vmax used in plt.imshow'''
    vpar=[0,0] #array to store vmin and vmax values for plt.imshow
    unr_imgdata=unravel(imgdata) 
    #Using np.quantile to get the 1st and 98.5th percentile of data
    vpar[0]=np.quantile(unr_imgdata,0.01) 
    vpar[1]=np.quantile(unr_imgdata,0.985)
    plt.figure(figsize=(10, 10))  
    plt.subplot(projection=w)
    plt.imshow(image_data,cmap='grey',vmin=vpar[0],vmax=vpar[1])  #displaying in greyscale, with the 1st and 98.5th percentile data as upper and lower limits
    plt.colorbar()  
    plt.title('Plot of NGC 104 with axes as sky coordinates')  
    plt.xlabel('RA(deg)')
    plt.ylabel('Dec(deg)')
    plt.show()
    return vpar

In [ ]:
def create_bsdata(arr):
    '''This function converts the image data into background subtracted data by masking the nan rows then subtracting the median of the data.
    It takes the image data array as an input parameter and returns the background subtracted data with nan rows set to 0.
    Example: If arr=[ nan -0.0403614  -0.03563905 nan nan
                        -0.02530874 -0.01923032 -0.03322243 -0.0152488  -0.02228408
                        nan nan nan nan nan]
    The array returned is [0.0 -0.01505266 -0.01033031 0.0 0.0
                            0.0 0.00607842 -0.00791369 0.01005994 0.00302466
                            0.0 0.0 0.0 0.0 0.0]'''  
    arr_ma=np.ma.masked_array(arr,np.isnan(arr)) #masking nan rows
    arr_ma_med=np.ma.median(arr_ma)
    arr=arr-arr_ma_med
    arr[np.isnan(arr)]=0.0 #setting nan rows to 0
    return arr

In [ ]:
def calc_std(arr):
    '''This function calculates the standard deviation of an array taken as an input parameter. 
    It masks the nan values and uses the numpy standard deviation function to calculate the standard deviation of the array. '''
    arr_ma=np.ma.masked_array(arr,np.isnan(arr))
    arr_ma_std=np.ma.std(arr_ma)
    return arr_ma_std

In [ ]:
def findstars(imgdata,vpar,plot):
    '''This function is used to detect stars in the image by making use of the DAOStarFinder class in the astropy.photoutils package. It has the following input parameters:
    1. imgdata- This is the image data from the FITS file.
    2. vpar- The values of vmin and vmax used in plt.imshow. This is obtained from the wcsplot function.
    3. plot- This parameter is used as a flag to plot the detected stars or not. 
    Plotting all the detected stars in an image is quite time consuming and the plot itself is quite clustered, so I choose to plot only a portion of the image with the detected stars.
    Running the function with plot not equal to 1 lets us find all the stars in the image without plotting everything. 
    The function returns a table with information about the detected stars, the most important of which (to us) is the x and y centroid values.'''
    std=calc_std(np.ravel(imgdata))
    bsdata=create_bsdata(np.ravel(imgdata))
    findata=bsdata.reshape(*imgdata.shape) #bsdata is a flattened (1D) array, this converts it back to the original shape
    daofind=photutils.detection.DAOStarFinder(fwhm=3.0,threshold=0.1*std)  #this value of standard deviation yields around 60000 stars
    sources=daofind(findata)
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=5.0)
    if(plot==1):
        plt.figure(figsize=(10,10))
        plt.imshow(imgdata,cmap='grey',origin='lower',vmin=vpar[0],vmax=vpar[1])
        apertures.plot(color='red', lw=1.5, alpha=0.5)
        plt.title('Zoomed in image of the NGC 104 cluster along with detected star positions')
        plt.show()
    return sources

In [ ]:
def gauss_bgadded(x,A,mean,sigma,bg):
    '''This function is a 1D gaussian function used to model the histograms of X and Y positions of detected stars. It takes the following input parameters:
    1. x- 1D array of the data used to create the gaussian function, here X or Y positions
    2. A- Maximum height of the gaussian
    3. mean- Mean of input data x
    4. sigma- Standard deviation of x
    5. bg- Term to account for the constant background level'''
    return A*np.exp(-0.5*((x-mean)/sigma)**2)+bg   

In [ ]:
def fitted_hist(posdata,bins,label,title):
    '''This function creates a histogram of the image data (rather, the positions of stars detected from the data) and fits it to a gaussian function.
    The 2 are simultaneously plotted. The input parameters:
    1. posdata: This is the X or Y centroid data of stars detected in the image.
    2. bins: Number of bins to be used for the histogram.
    3. label: Label for X axis, since that depends on whether X or Y data is being fitted.
    4. title: Plot title
    Example: fitted_hist(stars['xcentroid'],100,'X Positions','Histogram along with fitted Gaussian for X positions') creates a 
    histogram fitted with a gaussian for the X centroid data. The histogram uses 100 bins and the X axis label and plot title are given by 
    the last 2 parameters.'''
    hist,edges=np.histogram(posdata,bins)
    histcentres=(edges[1:]+edges[:-1])/2 #center of a bin is the midpoint of the edges
    histmean=np.mean(histcentres) #value of mean for the gaussian
    histstd=np.std(histcentres) #value of sigma for the gaussian
    histmax=np.max(hist) #peak value of gaussian
    bg=np.min(hist) #the additional term to account for background is the minimum value of the histogram
    params,cov=curve_fit(gauss_bgadded,histcentres,hist,p0=(histmax,histmean,histstd,bg))
    fit=gauss_bgadded(histcentres,params[0],params[1],params[2],params[3])
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(posdata, bins=bins, alpha=0.5, label=label)
    plt.plot(histcentres, fit, 'r-', label="Fitted Gaussian")
    plt.xlabel(label)
    plt.ylabel("Frequency")
    plt.legend()
    plt.title(title+',number of bins='+str(bins))
    plt.show()

In [ ]:
def stellar_density_profile(xpos, ypos, pixel_arcsize=0.04, maxr=2000, bins=100):
    '''This function calculates the stellar density profile of the star cluster. It takes the following input parameters:
    1. xpos: A 1D array, here we give the array of X centroid data of stars detected.
    2. ypos: Same as above, but for Y centroid data.
    3. pixel_arcsize: Size of a pixel in arc seconds. 
    The UVIS detector pixel size corresponds to 0.04 arcsec per pixel so we set this as default value.
    4. maxr: Maximum radius limit in order to not include edges of the image as this distorts the profile. 
    The default value is 2000 since that is the size of the image.
    5. bins: Number of bins used for histogram, default value is 100. 
    The function creates and plots the stellar density profile and returns the arrays with bin centers, stellar density values and error bars
    for the histogram.'''
    pixelr = np.sqrt((xpos-2060.5)**2 + (ypos-2265.5)**2) 
    #radius = sqrt(X^2+Y^2), here we subtract the pixel coordinates of the center since the origin is not at (0,0)
    hist, edges = np.histogram(pixelr, bins=bins, range=(0, maxr))
    
    widths = np.diff(edges) #bin size= difference between positions of consecutive edges
    centers = (edges[:-1] + widths / 2)*pixel_arcsize #calculating centers of histogram bins in units of arcsec
    areas = np.pi * (edges[1:]**2 - edges[:-1]**2)
    density = hist / areas  #stellar density at a point is the histogram value at a bin/area of bin
    error_bars = np.sqrt(hist) / areas 

    plt.figure(figsize=(8, 6))
    plt.errorbar(centers, density, yerr=error_bars, fmt='o', markersize=4)
    plt.xscale("log")
    plt.yscale("log") #plotting in log-log scale
    plt.xlabel("Radius (arcsec)")
    plt.ylabel("Stellar Density (stars/arcsec^2)")
    plt.title("Stellar Density Profile")
    plt.grid(True)
    plt.show()

    return centers, density, error_bars

In [ ]:
def king_model(radius,a,gamma,n0):
    '''This function is a king model function used for globular clusters. The input parameters are:
    1. radius: Radius of the cluster at a point, this is taken in the form of an array of histogram centers. 
    2. a: Radial scale length.
    3. gamma: Power law index.
    4. n0: Normalization at zero radius.
    The function returns the king model profile for the parameters inputted and is used to model the stellar density profile. '''
    return n0/((1 + (radius/a)**2)**(gamma/2)) 

In [ ]:
def sd_fit(centers,density,errorbars):
    '''This function is used to fit the stellar density profile. It uses the curve_fit function of the scipy.optimize package. 
    The input parameters are the histogram bin centers, stellar density and errorbars returned by stellar_density_profile. 
    The function obtains the optimal parameters for the stellar density profile fitted with the king model and plots them simultaneously.
    It also obtains the core radius using the optimal values found.'''
    params,covariance = curve_fit(king_model, centers, density, sigma=errorbars) #the histogram errorbars is used as the sigma parameter of curve_fit
    a,gamma,norm=params
    rc=a*np.sqrt(2**(2/gamma)-1)
    plt.figure(figsize=(8, 6))
    plt.errorbar(centers, density, yerr=errorbars, fmt='o', markersize=4, label="Data")
    plt.plot(centers, king_model(centers, a, gamma, norm), 'r-', label="Best Fit")
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel("Radius (arcsec)")
    plt.ylabel("Stellar Density (stars/arcsec^2)")
    plt.title("Stellar Density Profile with King Model Fit")
    plt.grid(True)
    plt.legend()
    plt.show()
    print("Radius of the globular cluster is",rc,"arcseconds")

In [ ]:
v=[0,0]
fitsimg = fits.open('C:\Sem1\Period1\Prog4AA\ic2r02050_drz.fits')
image_data=fitsimg[1].data
w=WCS(fitsimg[1].header)
v=wcsplot(image_data,w)
print('\n')
stars=findstars(image_data,v,0) #finding the positions of stars in the image (but not plotting the whole image)
substars=findstars(image_data[3000:3500,3000:3500],v,1)  #plotting a small portion of the image along with the detected star positions
print('\n')
#creating histograms of X and Y positions, fitted to a gaussian function
fitted_hist(stars['xcentroid'],100,'X Positions','Histogram along with fitted Gaussian for X positions') 
print('\n')
fitted_hist(stars['ycentroid'],100,'Y Positions','Histogram along with fitted Gaussian for Y positions')
print('\n')
bin_centers, density, error_bars = stellar_density_profile(stars['xcentroid'],stars['ycentroid']) 
print('\n')
sd_fit(bin_centers,density,error_bars)